This notebook illustrates how the DESY ZTF alert archive can be accessed through the convenience methods in `ampel_notebook_utils.py`. The full archive API documentation can be found at:
https://ampel.zeuthen.desy.de/api/ztf/archive/docs

Note that access is controlled through the personal archive token (see below), and the data returned might hence vary from person to person.

In [ ]:
import os
from ampel_notebook_utils import * 

In [ ]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://response = requests.get(endpoint, headers=header )ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
archivetoken = os.environ["ARCHIVE_TOKEN"]

In [ ]:
# ZTF name of transients to explore
snname = "ZTF22aaylnhq"

#### Accessing raw ZTF alerts.

Will return a list of dictionaries, each containing alert information for one detection. 
Cutouts will have been removed and the `prv_candidates` field left empty (as all photometry is contained in the list).

In [ ]:
alerts = api_name2alerts(snname, archivetoken)

In [ ]:
print(f'Found {len(alerts)} ZTF detections.')

#### Return alert candidate ids.

Will return a list of alert candidate ids.

In [ ]:
candids = api_name2candids(snname, archivetoken)

In [ ]:
print(f'First detected in alert candid {min(candids)}')

As a version of the above, we can directly get the latest alert.

In [ ]:
api_name2candid(snname, archivetoken)

#### Return a `LightCurve` object

Will obtain the latest photometry for the SN, and convert this into a ZTF `LightCurve` object, which contains the ZTF alert information in the `photopoints` and `upperlimits` lists (under the `body` field). A `LightCurve` object contains all information present in the Ampel DB corresponding to a particular transient _state_ and acessible to the active user. 

Ampel T2 (and T3) units are provided `LightCurve` objects for each transient. 

In [ ]:
from ampel.view.LightCurve import LightCurve

In [ ]:
lightCurve = api_get_lightcurve(snname, archivetoken)

In [ ]:
lightCurve

Similarly, we can obtain a list of lightcurves for transient, each corresponding to one speficic state (i.e. information accessible at some point in time).

In [ ]:
lcs = api_get_lightcurves(snname, archivetoken)

In [ ]:
print(f'Found {len(lcs)} object LightCurves.')

#### Working with LightCurve data

LightCurve objects contain several convenience methods for retrieving subsets of the available information. The following cells show an example for doing a crude plot.

In [ ]:
gphot = lightCurve.get_ntuples(['jd', 'magpsf'], 
                              {'attribute': 'fid', 'operator': '==', 'value': 1})
rphot = lightCurve.get_ntuples(['jd', 'magpsf'], 
                              {'attribute': 'fid', 'operator': '==', 'value': 2})

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot( [p[0] for p in gphot], [p[1] for p in gphot], 'o', color='g', label='ZTF g')
plt.plot( [p[0] for p in rphot], [p[1] for p in rphot], 'o', color='r', label='ZTF r')
plt.gca().invert_yaxis()
plt.xlabel('JD')
plt.ylabel('ZTF mag')
_ = plt.legend()